In [14]:
# 导入Pandas包
import pandas as pd

# 1、熟练使用Pandas进行数据连接
# 2、熟练使用Pandas进行合并数据集

In [15]:
# 加载多份数据
df1 = pd.read_csv('../data/concat_1.csv')
df2 = pd.read_csv('../data/concat_2.csv')
df3 = pd.read_csv('../data/concat_3.csv')

In [16]:
# 使用concat函数对多个DataFrame进行连接
concat_df = pd.concat([df1, df2, df3])
concat_df

In [29]:
# 创建一个Python字典进行添加数据
data_dict = {'A':'n1', 'B':'n2', 'C':'n3', 'D' : 'n4'}
pd.concat([df1, pd.DataFrame([data_dict])])

In [34]:
# 添加数据列（认值是index 按行添加）
concat_columns = pd.concat([df1, df2, df3], axis = 1)

In [35]:
# 通过列名获取子集
concat_columns['A']

In [36]:
# 向DataFrame中添加一列（方法一）
concat_columns['new_column'] = ['n1', 'n2', 'n3', 'n4']
concat_columns

In [39]:
# 向DataFrame中添加一列（方法二）
new_columns_series  = ['cs1', 'cs2', 'cs3', 'cs4']
concat_columns['new_column_series'] = pd.Series(new_columns_series)
concat_columns

In [40]:
# 合并数据并重置索引，或许有序索引
pd.concat([df1, df2, df3], axis = 'columns', ignore_index = True)

In [48]:
# 使用concat按照列索引进行合并

# 1、改变原本数据行索引
df1.columns = ['A', 'B', 'C', 'D']
df2.columns = ['E', 'F', 'G', 'H']
df3.columns = ['A', 'C', 'F', 'H']

# 2、打印改变后的列索引数据
print(df1)
print(df2)
print(df3)

# 3、使用concat函数对数据列拼接，）
print(pd.concat([df1, df2, df3]))

# 这里的参数 join可以选择连接方式，内连接只能包含部分数据（{'inner', 'outer'}, default 'outer'
print(pd.concat([df1, df3], join = 'inner'))


In [54]:
# 使用concat连接具有不同行索引的数据

# 1、重置数据行索引
df1.index = [0, 1, 2, 3]
df2.index = [4, 5, 6, 7]
df3.index = [0, 2, 5, 7]

# 2、打印改变后的行索引数据
print(df1)
print(df2)
print(df3)

# 3、按照行索引进行拼接
print(pd.concat([df1, df2, df3], axis='columns'))

# 4、使用join参数，只保留索引匹配的结果
print(pd.concat([df1, df3], axis=1, join='inner'))


In [55]:
# 合并多个数据集

# 1、安装sqlalchemy包
!pip install sqlalchemy

In [68]:
# 1、导入sqlalchemy包
from sqlalchemy import create_engine

# 2、加载一个db文件的引擎
engine = create_engine('sqlite:///../data/chinook.db')

In [66]:
#  读取表tracks（从数据库中读取表，第一个参数是表名，第二个参数是数据库连接对象）
tracks = pd.read_sql_table('tracks', engine)
tracks.head()

In [69]:
# 读取genres表
genres = pd.read_sql_table('genres', engine)
genres

In [70]:
# 数据一对一合并
tracks_subset = tracks.loc[[0,62,76,98,110,193,204,281,322,359],]
tracks_subset

In [71]:
subset_dict = ['TrackId','GenreId', 'Milliseconds']
tracks_subset[subset_dict]



In [72]:
# 合并数据（左连接）
genres.merge(tracks_subset, on = 'GenreId', how = 'left')

In [73]:
# 合并数据（右连接）
genres.merge(tracks_subset, on = 'GenreId', how = 'right')

In [79]:
# 多对一合并

# 1、读取客户表指定列
customers_col = ['CustomerId', 'FirstName', 'LastName']
customers = pd.read_sql_table('customers', engine, columns = customers_col)


# 2、读取发票数据
invoices_col = ['InvoiceId', 'CustomerId']
invoices = pd.read_sql_table('invoices', engine, columns = invoices_col)

# 3、读取发票详情数据
invoice_items_col = ['InvoiceId', 'UnitPrice', 'Quantity']
invoice_items = pd.read_sql_table('invoice_items', engine, columns = invoice_items_col)

In [80]:
# 根据关联ID合并三张表数据
customer_info = customers.merge(invoices, on = 'CustomerId').merge(invoice_items, on = 'InvoiceId')
customer_info

In [84]:
# 计算用户每笔消费的总金额（assign方法 创建新列）
cust_inv = customer_info.assign(Total = customer_info['Quantity'] * customer_info['UnitPrice'])
cust_inv.head()


In [93]:
# 按照用户Id，姓名分组，分组后对总金额求和，并排序
group_by_dict = ['CustomerId', 'FirstName', 'LastName']
group_by_cust = cust_inv.groupby(group_by_dict)
group_by_cust_sum = group_by_cust['Total'].sum()
group_by_cust_sum.sort_values(ascending = False)


In [94]:
# 总结：concat和merge区别
# concat：
#   1、Pandas函数
#   2、可以垂直和水平连接两个或者多个Pandas对象
#   3、只使用索引对齐
#   4、默认是外连接也可以需改成内连接
# merge：
#   1、DataFrame方法
#   2、只能水平连接两个DataFrame对象
#   3、对齐是靠被调用的DataFrame的列或者行索引和另一个DataFrame的列或者行索引
#   4、默认是内连接（也可以是左连接、外连接、右连接）

